In [1]:
#!conda install -y html5lib

In [2]:
import lxml
import html5lib
import pandas as pd
import requests
from bs4 import BeautifulSoup

## Scrape the Data Table
This section scrapes the data from wikipedia.
Beautiful Soup is used to extract the table structure, which is then fed to Pandas to create the dataframe

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
request = requests.get(url)
soup = BeautifulSoup(request.content, 'html.parser')
tables = soup.find_all('table')
df = pd.read_html(str(tables[0]), header=0)


In [4]:
df[0].head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Clean the data
- Ignore cells with borough "not assigned"
- This is done by creating a boolean selector and creating a new dataframe using the inverse

In [5]:
na_rows = df[0].Borough == "Not assigned"
f0 = df[0][~na_rows]


- If Neighbourhood is not assigned then take the borough


In [6]:
pd.set_option('mode.chained_assignment', None)
nb_rows = f0.Neighbourhood == "Not assigned"
f0.loc[nb_rows, 'Neighbourhood'] = f0.loc[nb_rows, 'Borough']
pd.set_option('mode.chained_assignment', 'warn')

- Combined common postcodes. One row with neighbourhoods comma separated

First we use the grouby by function to group rows with the same postcode.
We create a blank dataframe, with the same columns, ready to receive the grouped data

In [7]:
grouped = f0.groupby('Postcode')
ndf = pd.DataFrame(columns=f0.columns)


We loop over each group, extracting the postcode, the Borough, and joining the Neighbourhood names.
Then we append to our new dataframe.


In [8]:
for postcode, group in grouped:
    g = {}
    g['Postcode'] = postcode
    g['Borough'] = group.Borough.iloc[0]
    g['Neighbourhood'] = ",".join(group['Neighbourhood'].values.tolist())
    ndf = ndf.append(g, ignore_index=True)


Finally we print the shape of the new dataframe

In [9]:
print(ndf.shape)


(103, 3)


# Part Two - Merge the Long Lat data
Read tghe longitude and lattitude for each postcode held in the CSV as a new data frame

In [16]:
ll_df = pd.read_csv("./Geospatial_Coordinates.csv")
print(ll_df.head())


  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


To combine the tables, we need to do a *join*.

Because the column names are not quite the same, we have to specify the join columns separately for left and right

This creates a new dataframe

In [15]:
j = ndf.merge(ll_df,left_on='Postcode', right_on='Postal Code')
print(j.head())

  Postcode      Borough                         Neighbourhood Postal Code  \
0      M1B  Scarborough                         Rouge,Malvern         M1B   
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union         M1C   
2      M1E  Scarborough       Guildwood,Morningside,West Hill         M1E   
3      M1G  Scarborough                                Woburn         M1G   
4      M1H  Scarborough                             Cedarbrae         M1H   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.773136 -79.239476  
